In [4]:
import pandas as pd

movies = pd.read_csv('hutto_movies.csv')
movies.set_index(['id'], inplace = True) 
print(movies.head().round(2))
movies.describe().round(2)

    sentiment                                               text
id                                                              
1        2.27  The Rock is destined to be the 21st Century's ...
2        3.53  The gorgeously elaborate continuation of ''The...
3       -0.60                     Effective but too tepid biopic
4        1.47  If you sometimes like to go to the movies to h...
5        1.73  Emerges as something rare, an issue movie that...


,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [0]:
import pandas as pd
pd.set_option('display.width', 75)

from nltk.tokenize import casual_tokenize 
bags_of_words = []

from collections import Counter
for text in movies.text:
    bags_of_words.append(Counter(casual_tokenize(text)))
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
print(df_bows.shape)
print(df_bows.head())
df_bows.head()[list(bags_of_words[0].keys())]

(10605, 20756)
   The  Rock  is  destined  to  be  the  21st  Century's  new  ...  Ill  \
0    1     1   1         1   2   1    1     1          1    1  ...    0   
1    2     0   1         0   0   0    1     0          0    0  ...    0   
2    0     0   0         0   0   0    0     0          0    0  ...    0   
3    0     0   1         0   4   0    1     0          0    0  ...    0   
4    0     0   0         0   0   0    0     0          0    0  ...    0   

   slummer  Rashomon  dipsticks  Bearable  Staggeringly  ’  ve  \
0        0         0          0         0             0  0   0   
1        0         0          0         0             0  0   0   
2        0         0          0         0             0  0   0   
3        0         0          0         0             0  0   0   
4        0         0          0         0             0  0   0   

   muttering  dissing  
0          0        0  
1          0        0  
2          0        0  
3          0        0  
4          0     

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Schwarzenegger,",",Jean,Claud,Van,Damme,or,Steven,Segal,.
0,1,1,1,1,2,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb = nb.fit(df_bows, movies.sentiment > 0)
movies['predicted_sentiment'] = nb.predict(df_bows) * 8 - 4
movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs()
print(pd.Series(movies.error.mean()).round(1))
movies['sentiment_ispositive'] = (movies.sentiment > 0).astype(int)
movies['predicted_ispositive'] = (movies.predicted_sentiment > 0).astype(int)
print((movies.predicted_ispositive ==  movies.sentiment_ispositive).sum() / len(movies))
movies['''sentiment predicted_sentiment sentiment_ispositive predicted_ispositive'''.split()].head(8)

0    2.4
dtype: float64
0.9344648750589345


,sentiment,predicted_sentiment,sentiment_ispositive,predicted_ispositive
id,,,,
1,2.266667,4,1,1
2,3.533333,4,1,1
3,-0.600000,-4,0,0
4,1.466667,4,1,1
5,1.733333,4,1,1
6,2.533333,4,1,1
7,2.466667,4,1,1
8,1.266667,-4,1,0


In [0]:
products = pd.read_csv('hutto_products.csv')
bags_of_words = []
for text in products.text:
    bags_of_words.append(Counter(casual_tokenize(text)))
df_product_bows = pd.DataFrame.from_records(bags_of_words)
df_product_bows = df_product_bows.fillna(0).astype(int)
df_all_bows = df_bows.append(df_product_bows)
df_all_bows = df_all_bows.fillna(0).astype(int)
print(df_all_bows.columns)
df_product_bows = df_all_bows.iloc[len(movies):][df_bows.columns]
print(df_product_bows.shape)
print(df_bows.shape)
products['ispos'] = (products.sentiment > 0).astype(int)
products['predicted_ispositive'] = nb.predict(df_product_bows).astype(int)
print((products.predicted_ispositive == products.ispos).sum() / len(products))
products.head()

Index(['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st',
       'Century's', 'new',
       ...
       'sligtly', 'owner', '81', 'defectively', 'warrranty', 'expire',
       'expired', 'voids', 'baghdad', 'harddisk'],
      dtype='object', length=23302)
(3546, 20756)
(10605, 20756)
0.5572476029328821


,id,sentiment,text,ispos,predicted_ispositive
0,1_1,-0.90,troubleshooting ad-2500 and ad-2600 no picture...,0,0
1,1_2,-0.15,"repost from january 13, 2004 with a better fit...",0,0
2,1_3,-0.20,does your apex dvd player only play dvd audio ...,0,0
3,1_4,-0.10,or does it play audio and video but scrolling ...,0,0
4,1_5,-0.50,before you try to return the player or waste h...,0,0
